In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("alphabay")

create_database('blp_dataset')
create_table('products_tmp_alphabay')

In [ ]:
images = []
p_img_hashes = {}

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            if "listing" in file and "listings.php_ac" not in file and "listingnew.php" != file and "listings.php" != file:
                page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

                if page_content is not None:
                    ## GET DESCRIPTION
                    description = ""
                    desc_content = page_content.find("div", attrs={"id": "div_content1"})

                    if desc_content is not None:
                        description = get_content(desc_content)
                        description = description.replace("Product Description ", "")

                    ## GET TITLE
                    title = ""
                    title_content = page_content.find("h1", attrs={"class": "normal"})

                    if title_content is not None:
                        title = get_content(title_content)
                    

                    if description != "" or title != "":
                        # Insert product into database
                        query = "INSERT INTO products_tmp_alphabay (name, description, market_name, has_image, illegal) VALUES (%s, %s, %s, %s, %s)"
                        values = (title, description, "Aphabay", "no_image", "t")

                        cursor.execute(query, values)
                        conn.commit()

                        product_id = cursor.lastrowid


                        ## GET IMAGE
                        raw_name = page_content.find("img", attrs={"class": "listing"})
                        
                        if raw_name is not None:
                            try:
                                name = raw_name["src"]

                                if name is not "":
                                    url_image = Path(f"{folder_image}\{name}")
                                    
                                    if url_image.exists():
                                        hash = imagehash.phash(Image.open(url_image))

                                        if hash not in p_img_hashes:
                                            print(f"Image {product_id} exists... creating")

                                            dst_dir = Path(f"D:\images\_alphabay\{product_id}.jpg")
                                            copyfile(url_image, dst_dir)
                                            update_image(product_id, "has_image")

                                            p_img_hashes[hash] = name
                                        else:
                                            print(f"Image {product_id} repeated. Skipping...")

                                    else:
                                        print(f"Image {product_id} does not exists")
                                else:
                                    print(f"Image {product_id} has no name")
                            except IndexError:
                                print(f"Image {product_id} has no src")